In [1]:
%load_ext lab_black

In [2]:
import re
import json

from django.db import connections

from rich import print

# Manual Stuff

* In wagtain/images/signal_handlers.py deactivate post_delete_file_cleanup, to avoid deleting original 🥶
* Start only postgres and then: dropdb homepage && createdb homepage && python manage.py migrate
* Sometimes after convert: python manage.py fixtree
* Sometimes, you have to run: python manage.py sqlsequencereset cast + execute via pgcli

# Fetch Legacy Data from Database Restore

In [7]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [dict(zip(columns, row)) for row in cursor.fetchall()]


class Legacy:
    def __init__(self, db_name="legacy"):
        self.db_name = db_name
        self.users = self.fetch_rows("select * from users_user")
        self.blogs = self.fetch_rows("select * from cast_blog")
        self.posts = self.fetch_rows("select * from cast_post")
        self.images = self.fetch_rows("select * from cast_image")
        self.galleries = self.fetch_rows("select * from cast_gallery")
        self.gallery_images = self.fetch_rows("select * from cast_gallery_images")
        self.videos = self.fetch_rows("select * from cast_video")
        self.audios = self.fetch_rows("select * from cast_audio")

    def fetch_rows(self, stmt):
        with connections[self.db_name].cursor() as cursor:
            cursor.execute(stmt)
            rows = dictfetchall(cursor)
        return rows


class Converter:
    def __init__(self, legacy):
        self.legacy = legacy
        self.blog_content_type = ContentType.objects.get(app_label="cast", model="blog")

    def users(self):
        for l_user in self.legacy.users:
            user = User(**l_user)
            user.save()
        return {user.pk: user for user in User.objects.all()}

    def blogs(self):
        blog_legacy_to_wagtail = {}
        root = Page.objects.get(title="Welcome to your new Wagtail site!")
        for l_blog in self.legacy.blogs:
            kwargs = l_blog.copy()
            del kwargs["user_id"]
            del kwargs["id"]
            kwargs["owner"] = self.user_lookup[l_blog["user_id"]]
            l_blog["content_type"] = self.blog_content_type
            blog = Blog(**kwargs)
            blog = root.add_child(instance=blog)
            blog_legacy_to_wagtail[l_blog["id"]] = blog.pk
        return blog_legacy_to_wagtail

    def images(self):
        for num, l_image in enumerate(self.legacy.images):
            image = Image(
                pk=l_image["id"],
                file=l_image["original"],
                uploaded_by_user=self.user_lookup[l_image["user_id"]],
                created_at=l_image["created"],
                width=l_image["original_width"],
                height=l_image["original_height"],
            )
            image.save()
            if num % 300 == 0:
                print(num)

    def galleries(self):
        for l_gallery in self.legacy.galleries:
            kwargs = {k: v for k, v in l_gallery.items() if k != "user_id"}
            gallery = Gallery(**kwargs)
            gallery.save()

    def gallery_image_links(self):
        links = [
            (gi["id"], gi["gallery_id"], gi["image_id"])
            for gi in self.legacy.gallery_images
        ]
        stmt = "insert into cast_gallery_images (id, gallery_id, image_id) values (%s, %s, %s)"
        with connections["default"].cursor() as cursor:
            cursor.executemany(stmt, links)

    def convert(self):
        self.user_lookup = self.users()
        self.blog_lookup = self.blogs()
        self.images()
        self.galleries()
        self.gallery_image_links()

In [8]:
converter = Converter(Legacy())

In [6]:
%%time
converter.convert()

0

300

600

900

1200

CPU times: user 4.37 s, sys: 512 ms, total: 4.88 s
Wall time: 4.3 s


In [34]:
#post = Post.objects.first()
#print(post.body._raw_data)

[
    {
        'type': 'overview',
        'value': [
            {'type': 'heading', 'value': 'Eine Überschrift', 'id': '58378935-196d-4396-a23e-2bc1c9cbc0f3'},
            {
                'type': 'paragraph',
                'value': '<p data-block-key="anlsf">Ein bisschen Text. Angeblich geht hier auch RichText. Hmm, wäre
das dann eine Überschrift?</p><p data-block-key="r0vs"></p><p data-block-key="7pj4l"><a 
href="http://wersdoerfer.de">wersdoerfer</a>Aha!</p><p data-block-key="bivd5"></p><h2 
data-block-key="6vmjl">Heading 2</h2>',
                'id': '9279543d-88aa-4be3-a12c-952ebe6966c0'
            }
        ],
        'id': 'bdab0f1c-c164-417b-879d-d4c6583c4c6c'
    },
    {
        'type': 'detail',
        'value': [
            {'type': 'heading', 'value': 'Detail Überschrift', 'id': 'd7cbf386-58af-4ba6-bf7b-091e2304578a'},
            {
                'type': 'paragraph',
                'value': '<p data-block-key="g3l0n">Noch ein bisschen Text :)</p>',
                'id': 'f43fc4f3-f34f-4506-9a23-1ef4564de0e7'
            },
            {'type': 'image', 'value': 4, 'id': '45993490-7907-4d11-ac90-35bbe87f81ee'}
        ],
        'id': '6e18b320-7cfa-4255-9226-35b52b93aaac'
    }
]

In [33]:
def is_tag(text):
    return text.startswith("{%") and text.endswith("%}")


def tag_to_block(tag):
    tag_name, tag_id = tag.strip("{%").strip("%}").split()
    tag_id = int(tag_id)
    if tag_name == "gallery":
        image_blocks = []
        for image in Gallery.objects.get(pk=184).images.all():
            image_blocks.append({"type": "item", "value": image.pk})
        return {"type": tag_name, "value": image_blocks}
    return {"type": tag_name, "value": tag_id}


def content_to_streamfield(content):
    blocks = overview = []
    just_tag = re.compile(r"({% \w+ \d+ %})")
    for part in just_tag.split(content):
        if len(part) == 0:
            continue
        if is_tag(part):
            blocks.append(tag_to_block(part))
        else:
            blocks.append({"type": "paragraph", "value": part})
    return [{"type": "overview", "value": overview}]


def build_post_from_legacy(legacy):
    # post = Post(title=legacy["title"], visible_date=legacy["visible_date"])
    kwargs = {
        k: v
        for k, v in legacy.items()
        if k not in ["content", "author_id", "blog_id", "id"]
    }
    kwargs["content_type"] = ContentType.objects.get(app_label="cast", model="post")
    print(kwargs)
    post = Post(**kwargs)
    raw_body = content_to_streamfield(legacy["content"])
    post.body = json.dumps(raw_body)
    return post

In [34]:
blog = Page.objects.get(slug="ephes_blog")

In [35]:
[l_post] = [p for p in legacy.posts if p["id"] == 332]

In [36]:
post = build_post_from_legacy(l_post)

{
    'created': datetime.datetime(2022, 11, 1, 9, 27, 54, 587153, tzinfo=datetime.timezone.utc),
    'modified': datetime.datetime(2022, 11, 5, 7, 44, 32, 666170, tzinfo=datetime.timezone.utc),
    'title': 'Weeknotes 2022-10-31',
    'pub_date': datetime.datetime(2022, 11, 5, 7, 42, 29, tzinfo=datetime.timezone.utc),
    'visible_date': datetime.datetime(2022, 10, 31, 0, 0, tzinfo=datetime.timezone.utc),
    'slug': 'weeknotes-2022-10-31',
    'podcast_audio_id': None,
    'uuid': UUID('105b836b-514c-466b-ba72-6db24c63b9e3'),
    'keywords': '',
    'explicit': 1,
    'block': False,
    'comments_enabled': True,
    'content_type': <ContentType: cast | post>
}

In [48]:
print(post.body.raw_data)

[{'type': 'overview', 'value': [{'type': 'paragraph', 'value': '<p>Spent quite some time fiddling around with my 
phone and Computers because of the&nbsp;<a href="https://www.macrumors.com/guide/ios-16-1-features/">iOS 
16.1</a>&nbsp;/ <a href="https://www.macrumors.com/roundup/macos-13/">macOS 13</a>&nbsp;releases. Notes stopped 
syncing with my Macs (which happens with every major release). Reminders synced shared lists across different users
and phones (wow, that usually also breaks on new releases), but it stopped syncing with my watch, so I had&nbsp;to 
take out my phone and look at it during grocery shopping (like an animal). I also tried out the&nbsp;eagerly 
awaited shared iCloud photo library, and ran into a lot of edge cases/bugs (the&nbsp;image and&nbsp;video 
counts&nbsp;stopped being accurate for me a while ago, but now they are just going crazy:&nbsp;edited images are 
converted to&nbsp; tiff sometimes and then they cannot&nbsp;be uploaded to iCloud anymore and&nbsp;some images just
refuse to be added to the shared library, etc pp). Maybe I have just too many notes, reminders, and photos.<br 
/>\r\n<br />\r\nWorked a little bit on&nbsp;<a href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;to 
bring its dependencies up to date. There&#39;s a Wagtail&nbsp;<a 
href="https://github.com/wagtail/rfcs/blob/main/text/071-responsive-images-support.md">RFC 71: support for 
responsive images</a>&nbsp;that maybe makes my&nbsp;<a 
href="https://github.com/ephes/wagtail_srcset">wagtail-srcset</a>&nbsp;workaround obsolete. Would be cool if this 
is no longer needed. Planning to look into this in the next 
week.</p>\r\n\r\n<h3>Articles</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://thepointmag.com/examined-life/a-world-without-why/">A World Without Why?</a>&nbsp;| Achievable 
aesthetics over <meta charset="utf-8" />unachievable ethics, no?</li>\r\n\t<li><a 
href="https://lemire.me/blog/2022/10/16/science-and-technology-links-october-16-2022/">Science and Technology links
(October 16 2022)</a></li>\r\n\t<li><a 
href="https://www.theverge.com/2022/10/28/23428132/elon-musk-twitter-acquisition-problems-speech-moderation">Welcom
e to hell, Elon</a>&nbsp;| Interesting social experiment. I&#39;m curious about how this all will play out, but I 
wouldn&#39;t&nbsp;bet on the fediverse winning, sorry.</li>\r\n</ul>\r\n\r\n<h3><br 
/>\r\nTwitter</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://twitter.com/groove_sdc/status/1584951348730417153?s=12&amp;t=L0VXHhYpJyucgh9MGopXyA">Dear White 
People, No one is asking you to apologise for your ancestors. We are asking you to dismantle the systems of 
oppression they built, that you maintain and benefit from.</a>&nbsp;| Well, fair enough..</li>\r\n\t<li><a 
href="https://twitter.com/coomsicle/status/1586163302115463168?s=12&amp;t=d9Ry7_G5A6r0E43Q0W1v-g">i hate doing acid
with other people. bro over here bugging out talmbout &ldquo;is this really all there is&rdquo;. can you please 
shut up i&rsquo;m trying to hands-free cum</a></li>\r\n\t<li><a 
href="https://twitter.com/conorfi/status/1586277307869958145?s=12&amp;t=N2XF9P6PnxVSiP9m_CtBYg">3 rules: -no 
wishing for death -no falling in love -no bring back dead people. I want to use Excel as a database. There are 4 
rules.</a></li>\r\n</ul>\r\n&nbsp;\r\n\r\n<h3>Weeknotes</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://shezi.de/weeklogs/log-2022-10-23.html">Weeklog for Week 42: October 17 to October 23</a>&nbsp;| 
Johannes</li>\r\n</ul>\r\n&nbsp;\r\n\r\n<h3>Software</h3>\r\n\r\n<ul>\r\n\t<li><a 
href="https://textual.textualize.io/tutorial/">Welcome to the Textual Tutorial!</a>&nbsp;| Really well-done 
tutorial</li>\r\n\t<li><a href="https://aeturrell.github.io/coding-for-economists/vis-common-plots.html">Common 
Plots</a>&nbsp;| Lots of cool example plots in&nbsp;<a href="https://matplotlib.org/">matplotlib</a>,&nbsp;<a 
href="https://plotnine.readthedocs.io/en/stable/">plotnine</a>,&nbsp;<a 
href="https://seaborn.pydata.org/">seaborn</a>,&nbsp;

In [50]:
post = blog.add_child(instance=post)

NodeAlreadySaved: Attempted to add a tree node that is already in the database

In [56]:
def is_tag(text):
    return text.startswith("{%") and text.endswith("%}")


def tag_to_block(tag):
    tag_name, tag_id = tag.strip("{%").strip("%}").split()
    tag_id = int(tag_id)
    return {"type": tag_name, "value": tag_id}


def content_to_streamfield(content):
    blocks = overview = []
    just_tag = re.compile(r"({% \w+ \d+ %})")
    for part in just_tag.split(content):
        if len(part) == 0:
            continue
        if is_tag(part):
            blocks.append(tag_to_block(part))
        else:
            blocks.append({"type": "paragraph", "value": part})
    return [{"type": "overview", "value": overview}]

In [57]:
content_to_streamfield(l_post["content"])

[{'type': 'overview',
  'value': [{'type': 'paragraph',
    'value': '<p>Short work week. Gave me time to indulge in heavy yak shaving. At first, I tried to just make&nbsp;<a href="https://github.com/ephes/django-cast">django-cast</a>&nbsp;(the develop branch) work for newer <a href="https://www.djangoproject.com/">Django</a> and <a href="https://wagtail.org/">Wagtail</a> versions. Then I realized I had to fix&nbsp;<a href="http://github.com/ephes/wagtail_srcset/issues/2">wagtail-srcset</a>&nbsp;first and replaced&nbsp;<a href="https://python-poetry.org/">poetry</a>&nbsp;with&nbsp;<a href="https://flit.pypa.io/en/stable/">flit</a>&nbsp;while I&#39;m at it. Then I got lost fixing some stuff in&nbsp;<a href="https://github.com/ephes/kptncook/issues/18">kptncook</a>&nbsp;before I remembered that I have to also update&nbsp;<a href="https://github.com/ephes/django-indieweb">django-indieweb</a>&nbsp;and&nbsp;<a href="https://github.com/ephes/django_fileresponse/tree/main/">django-filerespons

In [45]:
l_post["content"][-20:]

'3>\r\n{% image 1281 %}'

In [29]:
post = blog.add_child(instance=post)

In [14]:
post.visible_date

datetime.datetime(2022, 11, 7, 0, 0, tzinfo=datetime.timezone.utc)

In [17]:
post.delete()

# Get Templatetags from Content

In [39]:
just_tag = re.compile(r"({% \w+ \d+ %})")
just_tag.split("foo {% asdf 34 %} bar")

['foo ', '{% asdf 34 %}', ' bar']

In [38]:
#split_tags.split(legacy["content"])

In [44]:
parse_tag = re.compile(r"{% (\w+) (\d+) %}")
[(tag_name, tag_pk)] = parse_tag.findall("{% asdf 34 %}")
print(tag_name, tag_pk)

asdf 34

In [42]:
parse_tag.findall("asdf")

[]

In [21]:
tags_regex.findall(legacy["content"])

[('image', '1281')]

In [51]:
tag_name, tag_id = "{% image 1281 %}".strip("{%").strip("%}").split()
tag_id = int(tag_id)
print(tag_name, tag_id)

image 1281

In [20]:
Page.objects.last().pk

6

In [30]:
print(Page.objects.get(slug="gallerie-test").post.body.raw_data)

[{'type': 'overview', 'value': [{'type': 'heading', 'value': 'asdf', 'id': 'a5b3c0cc-889b-4b57-bd81-3a4bed41528b'},
{'type': 'gallery', 'value': [{'type': 'item', 'value': 1281, 'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'}, 
{'type': 'item', 'value': 1280, 'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}], 'id': 
'2a30b103-f138-488d-aa73-713b1de7d5ef'}], 'id': '56fa286a-3d6e-46c1-8000-b29f0cc9e847'}]

In [44]:
{
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281, "id": "88c02e62-77e9-44f2-a84e-c3d84453cb13"},
        {"type": "item", "value": 1280, "id": "eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af"},
    ],
}

{'type': 'gallery',
 'value': [{'type': 'item',
   'value': 1281,
   'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'},
  {'type': 'item',
   'value': 1280,
   'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}]}

In [25]:
Page.objects.all()

<PageQuerySet [<Page: Root>, <Page: Welcome to your new Wagtail site!>, <Page: Claas sagt Hallo>, <Page: Ephes Blog>, <Page: Gallerie-Test>, <Page: sadf>]>

In [46]:
post.body.raw_data[0]["value"][-1]

{'type': 'gallery',
 'value': [{'type': 'item',
   'value': 1281,
   'id': '88c02e62-77e9-44f2-a84e-c3d84453cb13'},
  {'type': 'item',
   'value': 1280,
   'id': 'eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af'}]}

In [49]:
post.body.raw_data[0]["value"][-1] = {
    "type": "gallery",
    "value": [
        {"type": "item", "value": 1281, "id": "88c02e62-77e9-44f2-a84e-c3d84453cb13"},
        {"type": "item", "value": 1280, "id": "eb5d3c2e-6e47-434a-a18a-98e2e0c5d3af"},
    ],
}

In [51]:
Post.objects.all()

<PageQuerySet [<Post: Gallerie-Test>]>